In [23]:
# preprocessing/data_preprocessing.py

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import joblib

def preprocess_meteorological_data(file_path):
    # Load data
    met_data = pd.read_csv(file_path)

    # Combine DATE and MST into a single datetime column
    # met_data['Timestamp'] = pd.to_datetime(met_data['DATE'] + ' ' + met_data['MST'])
    # Combine DATE and MST into a single datetime column
    met_data['Timestamp'] = pd.to_datetime(met_data['datetime'].astype(str), format='%Y%m%d%H%M%S')

    # Sort data by Timestamp
    met_data.sort_values('Timestamp', inplace=True)

    # Handle missing values in input features using KNN imputation
    input_features = [
        'Tower Dry Bulb Temp [deg C]', 'Tower RH [%]', 'Station Pressure [mBar]',
        'Avg Wind Speed @ 6ft [m/s]', 'Avg Wind Direction @ 6ft [deg from N]'
    ]

    # Initialize KNN imputer
    imputer = KNNImputer(n_neighbors=5)

    # Fit and transform the input features
    met_data_imputed = imputer.fit_transform(met_data[input_features])

    # Update the DataFrame with imputed values
    met_data[input_features] = met_data_imputed

    # Handle missing values in the target variable separately
    target_variable = 'Global CMP22 (vent/cor) [W/m^2]'

    # Optionally interpolate missing target values
    # met_data[target_variable].interpolate(method='time', inplace=True)
    
    # Option 2: Drop rows with missing target values (uncomment if preferred)
    met_data.dropna(subset=[target_variable], inplace=True)

    # Rename columns for simplicity
    met_data.rename(columns={
        'Tower Dry Bulb Temp [deg C]': 'Temperature',
        'Tower RH [%]': 'Humidity',
        'Station Pressure [mBar]': 'Pressure',
        'Avg Wind Speed @ 6ft [m/s]': 'Wind Speed',
        'Avg Wind Direction @ 6ft [deg from N]': 'Wind Direction',
        'Global CMP22 (vent/cor) [W/m^2]': 'Irradiance'
    }, inplace=True)

    # Feature scaling for input features
    scaler = MinMaxScaler()
    met_data[['Temperature', 'Humidity', 'Pressure', 'Wind Speed']] = scaler.fit_transform(
        met_data[['Temperature', 'Humidity', 'Pressure', 'Wind Speed']]
    )
    joblib.dump(scaler, 'scaler_y.pkl')

    # Wind Direction encoding (convert degrees to sine and cosine components)
    met_data['Wind Dir Sin'] = np.sin(np.deg2rad(met_data['Wind Direction']))
    met_data['Wind Dir Cos'] = np.cos(np.deg2rad(met_data['Wind Direction']))
    met_data.drop('Wind Direction', axis=1, inplace=True)

    # Temporal features
    met_data['Hour'] = met_data['Timestamp'].dt.hour / 23.0  # Normalize Hour
    met_data['DayOfYear'] = met_data['Timestamp'].dt.dayofyear / 365.0  # Normalize DayOfYear

    # Prepare target variables (future irradiance)
    target = 'Irradiance'
    for minutes in [5, 15, 30, 60]:
        met_data[f'Irradiance_{minutes}min_ahead'] = met_data[target].shift(-minutes)

    # **Removed the line that drops rows with NaN values after shifting**
    # We will handle dropping NaN values after merging with images
    # met_data.dropna(inplace=True)

    # Reset index and return the processed DataFrame
    return met_data.reset_index(drop=True)

In [24]:
# preprocessing/image_preprocessing.py

import cv2
import numpy as np
import glob
import os
import pandas as pd

def preprocess_images(image_folder):
    image_paths = sorted(glob.glob(os.path.join(image_folder, '*.jpg')))
    images = []
    image_timestamps = []

    for path in image_paths:
        # Extract timestamp from image filename
        # Assuming filename format: YYYYMMDDHHMMSS.jpg
        filename = os.path.basename(path)
        timestamp_str = filename.replace('.jpg', '')
        timestamp = pd.to_datetime(timestamp_str, format='%Y%m%d%H%M%S')

        img = cv2.imread(path)
        if img is None:
            continue  # Skip if the image is not readable
        # Iterates over each image path, reads the image using OpenCV, and resizes it to 128x128 pixels
	    # Normalizes pixel values to the range [0, 1] by dividing by 255
        img = cv2.resize(img, (128, 128))
        img = img / 255.0  # Normalize pixel values
        images.append(img)
        image_timestamps.append(timestamp)

    return images, image_timestamps

In [25]:
met_data = preprocess_meteorological_data('weather_data.csv')
met_data

,datetime,Irradiance,Temperature,Humidity,Wind Speed,Pressure,image_url,Timestamp,Wind Dir Sin,Wind Dir Cos,Hour,DayOfYear,Irradiance_5min_ahead,Irradiance_15min_ahead,Irradiance_30min_ahead,Irradiance_60min_ahead
0,20241116065000,8.16102,0.221854,0.598923,0.311558,0.000000,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-16 06:50:00,-0.761538,0.648120,0.260870,0.879452,14.6895,31.3017,66.4804,152.526
1,20241116065100,9.39915,0.232479,0.573800,0.369447,0.003761,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-16 06:51:00,-0.814116,0.580703,0.260870,0.879452,16.1444,33.2346,69.2944,155.594
2,20241116065200,10.62110,0.241836,0.587707,0.273970,0.003448,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-16 06:52:00,-0.719340,0.694658,0.260870,0.879452,17.6113,35.2643,72.1972,158.274
3,20241116065300,11.92040,0.242129,0.595334,0.181508,0.003918,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-16 06:53:00,-0.846193,0.532876,0.260870,0.879452,19.0965,37.5257,74.9238,161.364
4,20241116065400,13.26040,0.240667,0.593540,0.176281,0.004701,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-16 06:54:00,-0.931056,0.364877,0.260870,0.879452,20.6639,39.6576,77.6095,164.285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,20241117081800,238.66300,0.403158,0.557649,0.344322,1.000000,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-17 08:18:00,0.705872,-0.708340,0.347826,0.882192,NaN,NaN,NaN,NaN
680,20241117081900,241.60100,0.403158,0.537909,0.238794,0.995299,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-17 08:19:00,0.653421,-0.756995,0.347826,0.882192,NaN,NaN,NaN,NaN
681,20241117082000,244.37500,0.399162,0.602961,0.294070,0.989500,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-17 08:20:00,0.746638,-0.665230,0.347826,0.882192,NaN,NaN,NaN,NaN
682,20241117082100,247.39600,0.397895,0.662180,0.334271,0.988246,https://solar-tracker-images.s3.us-east-2.amaz...,2024-11-17 08:21:00,0.721760,-0.692143,0.347826,0.882192,NaN,NaN,NaN,NaN


In [26]:
images, image_timestamps = preprocess_images('pics')
len(image_timestamps)

684

In [27]:
# Step 2: Align Data with Images
def align_data_with_images(met_data, images, image_timestamps):
    """
    Aligns meteorological data with corresponding images based on timestamps.

    Parameters:
    - met_data: pandas DataFrame containing meteorological data.
    - images: list or numpy array of preprocessed images.
    - image_timestamps: list or pandas Series of image timestamps.

    Returns:
    - merged_data: pandas DataFrame containing aligned meteorological data and images.
    """
    # Create a DataFrame for image timestamps
    image_df = pd.DataFrame({'Timestamp': image_timestamps, 'Image': images})
    image_df['Timestamp'] = pd.to_datetime(image_df['Timestamp'])
    
    # Ensure meteorological data has datetime objects
    met_data['Timestamp'] = pd.to_datetime(met_data['Timestamp'])
    
    # Merge meteorological data with images using an inner join to keep only matching timestamps
    merged_data = pd.merge(met_data, image_df, on='Timestamp', how='inner')
    
    # Prepare target variables (future irradiance)
    target = 'Irradiance'
    for minutes in [5, 15, 30, 60]:
        merged_data[f'Irradiance_{minutes}min_ahead'] = merged_data[target].shift(-minutes)
    
    # Drop rows with any remaining missing values (due to shifting)
    merged_data.dropna(inplace=True)
    
    # Reset index after dropping rows
    merged_data.reset_index(drop=True, inplace=True)
    
    return merged_data

merged_data = align_data_with_images(met_data, images, image_timestamps)

# Step 3: Split Data into Training and Testing Sets BEFORE Sequence Creation
split_ratio = 0.8  # 80% for training, 20% for testing
split_index = int(len(merged_data) * split_ratio)

# Perform the split
train_data = merged_data.iloc[:split_index]
test_data = merged_data.iloc[split_index:]

print(f"Training data contains {len(train_data)} records.")
print(f"Testing data contains {len(test_data)} records.")


Training data contains 499 records.
Testing data contains 125 records.


In [ ]:
# training/train_model.py

import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from models.hybrid import create_hybrid_model  # Ensure correct import path
import pandas as pd

def train_hybrid_model(train_data, sequence_length=60):
    """
    Trains the hybrid CNN-LSTM model on the provided training data.

    Parameters:
    - train_data: pandas DataFrame containing training meteorological data and images.
    - sequence_length: Number of past minutes to consider for each sequence.

    Returns:
    - model: Trained Keras model.
    - history: Training history object.
    """
    
    # Define feature columns
    features = [
        'Temperature', 'Humidity', 'Pressure', 'Wind Speed',
        'Wind Dir Sin', 'Wind Dir Cos', 'Hour', 'DayOfYear'
    ]  # Exclude direct current irradiance as a feature

    # Extract features and targets
    X_num_train, X_img_train, y_train = create_sequences(train_data, sequence_length, features)

    print(f"Created {len(X_num_train)} training sequences.")

    # Create and compile the model
    num_features = X_num_train.shape[2]
    model = create_hybrid_model(sequence_length, num_features)

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    # Define callbacks
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3)

    # Train the model with dictionary inputs
    history = model.fit(
        {'Image_Input': X_img_train, 'LSTM_Input': X_num_train},
        y_train,
        validation_split=0.1,  # Further split training data for validation
        epochs=50,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr]
    )

    return model, history

def create_sequences(data, seq_length, features):
    """
    Creates input sequences and corresponding targets from the data.

    Parameters:
    - data: pandas DataFrame containing the data.
    - seq_length: Length of each input sequence.
    - features: List of feature column names.

    Returns:
    - X_num_seq: Numpy array of numerical feature sequences.
    - X_img_seq: Numpy array of image sequences.
    - y_seq: Numpy array of target irradiance values.
    """
    X_num_seq, X_img_seq, y_seq = [], [], []
    for i in range(len(data) - seq_length):
        # Extract numerical features for the sequence
        X_num_seq.append(data[features].values[i:i+seq_length])
        
        # Extract the corresponding image at the last timestamp of the sequence
        X_img_seq.append(data['Image'].values[i+seq_length-1])
        
        # Extract the target irradiance values at the end of the sequence
        y_seq.append(data[[f'Irradiance_{minutes}min_ahead' for minutes in [5, 15, 30, 60]]].values[i+seq_length-1])
    
    return np.array(X_num_seq), np.array(X_img_seq), np.array(y_seq)




Created 439 training sequences.
Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['Image_Input', 'LSTM_Input']. Received: the structure of inputs={'Image_Input': '*', 'LSTM_Input': '*'}
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 159ms/step - loss: 166790.0938 - mae: 378.7816 - val_loss: 250638.4531 - val_mae: 494.5575 - learning_rate: 0.0010
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step - loss: 39544.9961 - mae: 166.5748 - val_loss: 163756.1562 - val_mae: 403.7149 - learning_rate: 0.0010
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - loss: 24068.4258 - mae: 130.7008 - val_loss: 102439.9688 - val_mae: 317.7763 - learning_rate: 0.0010
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - loss: 21915.4785 - mae: 130.7383 - val_loss: 108349.6797 - val_mae: 328.3203 - learning_rate: 0.0010
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step - loss: 20559.5840 - mae: 119.0342 - val_loss: 79231.0312 - val_mae: 280.2571 - learning_rate: 0.0010
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - loss: 69455.0391 - mae: 164.0488 - val_loss: 71176.6641 - val_mae: 263.1262 - learning_rate: 0.0010
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - loss: 16114.1777 - mae: 108.

In [29]:
# Step 4: Train the Model
model, history = train_hybrid_model(train_data, sequence_length=60)

# Save the model
model.save('trainedModels/test.keras')

Created 439 training sequences.
Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['Image_Input', 'LSTM_Input']. Received: the structure of inputs={'Image_Input': '*', 'LSTM_Input': '*'}
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 147ms/step - loss: 169494.0000 - mae: 380.9048 - val_loss: 175893.5469 - val_mae: 363.1328 - learning_rate: 0.0010
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - loss: 52095.5117 - mae: 188.7865 - val_loss: 118502.2812 - val_mae: 338.3320 - learning_rate: 0.0010
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - loss: 25500.9004 - mae: 139.2729 - val_loss: 102648.1328 - val_mae: 319.5043 - learning_rate: 0.0010
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - loss: 22112.6035 - mae: 126.4584 - val_loss: 75646.5312 - val_mae: 273.6306 - learning_rate: 0.0010
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - loss: 17687.1602 - mae: 118.6458 - val_loss: 60440.7148 - val_mae: 244.1346 - learning_rate: 0.0010
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - loss: 14382.2988 - mae: 105.5236 - val_loss: 50610.8906 - val_mae: 222.1486 - learning_rate: 0.0010
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - loss: 11719.6133 - mae: 93.67

In [43]:
# evaluation/evaluate_model.py

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime

def evaluate_model(model, test_data, sequence_length=60):
    """
    Evaluates the regression model on the provided test_data.
    Saves evaluation plots in a uniquely named subfolder within 'evaluation_plots'.
    
    Parameters:
    - model: Trained Keras model.
    - test_data: pandas DataFrame containing testing meteorological data and images.
    - sequence_length: Number of past minutes to consider for each sequence.
    """
    
    # -----------------------------
    # 1. Setup Directory for Saving Plots
    # -----------------------------
    
    # Define the base directory for evaluation plots
    base_dir = 'evaluation_plots'
    
    # Create the base directory if it doesn't exist
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        print(f"Created base directory for evaluation plots at '{base_dir}'.")
    
    # Generate a unique subfolder name using the current timestamp
    run_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    run_folder = os.path.join(base_dir, f'run_{run_timestamp}')
    
    # Create the subfolder
    os.makedirs(run_folder, exist_ok=True)
    print(f"Created run-specific directory at '{run_folder}'.")
    
    # -----------------------------
    # 2. Prepare Data for Evaluation
    # -----------------------------
    
    # Define feature columns
    features = [
        'Temperature', 'Humidity', 'Pressure', 'Wind Speed',
        'Wind Dir Sin', 'Wind Dir Cos', 'Hour', 'DayOfYear'
    ]  # Exclude direct current irradiance as a feature

    # Extract features and targets
    X_num = test_data[features].values
    y = test_data[[f'Irradiance_{minutes}min_ahead' for minutes in [5, 15, 30, 60]]].values
    X_img = np.array(test_data['Image'].tolist())

    # -----------------------------
    # 3. Create Sequences
    # -----------------------------
    
    def create_sequences(X_num, X_img, y, seq_length):
        """
        Creates input sequences for the model.
        
        Parameters:
        - X_num: Numpy array of numerical features.
        - X_img: Numpy array of images.
        - y: Numpy array of target variables.
        - seq_length: Length of the input sequences.
        
        Returns:
        - Tuple of Numpy arrays: (X_num_seq, X_img_seq, y_seq)
        """
        X_num_seq, X_img_seq, y_seq = [], [], []
        for i in range(len(X_num) - seq_length):
            X_num_seq.append(X_num[i:i+seq_length])
            X_img_seq.append(X_img[i+seq_length-1])  # Use image at the last timestamp
            y_seq.append(y[i+seq_length-1])
        return np.array(X_num_seq), np.array(X_img_seq), np.array(y_seq)

    # Generate sequences from test data
    X_num_seq, X_img_seq, y_seq = create_sequences(X_num, X_img, y, sequence_length)
    print(f"Created {len(X_num_seq)} sequences for evaluation.")

    # -----------------------------
    # 4. No Further Splitting Needed
    # -----------------------------
    
    # Since test_data is already separate, no need to split again
    X_num_test = X_num_seq
    X_img_test = X_img_seq
    y_test = y_seq
    
    print(f"Evaluation split: {len(X_num_test)} samples.")
    
    # -----------------------------
    # 5. Make Predictions
    # -----------------------------
    
    # Generate predictions using the trained model
    y_pred = model.predict({'Image_Input': X_img_test, 'LSTM_Input': X_num_test})
    print("Generated predictions for the test set.")

    # -----------------------------
    # 6. Calculate and Save Metrics and Plots
    # -----------------------------
    
    horizons = [5, 15, 30, 60]  # Prediction horizons in minutes
    
    # Initialize a text file to save metrics
    metrics_file = os.path.join(run_folder, 'metrics.txt')
    with open(metrics_file, 'w') as f:
        f.write("Evaluation Metrics:\n")
        f.write("===================\n\n")
    
    for i, minutes in enumerate(horizons):
        # Calculate metrics for each horizon
        rmse = np.sqrt(mean_squared_error(y_test[:, i], y_pred[:, i]))
        mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
        r2 = r2_score(y_test[:, i], y_pred[:, i])
        metric_str = f"{minutes}-Minute Ahead Prediction - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.2f}"
        print(metric_str)
        
        # Append metrics to the text file
        with open(metrics_file, 'a') as f:
            f.write(metric_str + "\n")
        
        # Plot Actual vs Predicted
        plt.figure(figsize=(10, 4))
        plt.plot(y_test[:, i], label='Actual', alpha=0.7)
        plt.plot(y_pred[:, i], label='Predicted', alpha=0.7)
        plt.title(f'{minutes}-Minute Ahead Prediction')
        plt.xlabel('Samples')
        plt.ylabel('Irradiance (W/m²)')
        plt.legend()
        plt.tight_layout()
        
        # Save the plot
        plot_filename = f'{minutes}_min_ahead_prediction.png'
        plot_path = os.path.join(run_folder, plot_filename)
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot: {plot_path}")
        
        # Plot Scatter of Actual vs Predicted
        plt.figure(figsize=(6, 6))
        plt.scatter(y_test[:, i], y_pred[:, i], alpha=0.5)
        plt.plot([y_test[:, i].min(), y_test[:, i].max()],
                 [y_test[:, i].min(), y_test[:, i].max()],
                 'r--', lw=2)
        plt.title(f'Actual vs Predicted Irradiance ({minutes} min Ahead)')
        plt.xlabel('Actual Irradiance (W/m²)')
        plt.ylabel('Predicted Irradiance (W/m²)')
        plt.tight_layout()
        
        # Save the scatter plot
        scatter_filename = f'actual_vs_predicted_{minutes}_min_ahead.png'
        scatter_path = os.path.join(run_folder, scatter_filename)
        plt.savefig(scatter_path)
        plt.close()
        print(f"Saved scatter plot: {scatter_path}")
    
    # -----------------------------
    # 7. Calculate and Save Overall Metrics and Plots
    # -----------------------------
    
    # Calculate overall performance metrics across all horizons
    overall_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    overall_mae = mean_absolute_error(y_test, y_pred)
    overall_r2 = r2_score(y_test, y_pred)
    overall_metric_str = f"Overall Performance - RMSE: {overall_rmse:.2f}, MAE: {overall_mae:.2f}, R²: {overall_r2:.2f}"
    print(overall_metric_str)
    
    # Append overall metrics to the text file
    with open(metrics_file, 'a') as f:
        f.write("\n" + overall_metric_str + "\n")
    
    # Plot Overall Actual vs Predicted
    plt.figure(figsize=(6,6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()],
             [y_test.min(), y_test.max()],
             'r--', lw=2)
    plt.title('Actual vs Predicted Irradiance (Overall)')
    plt.xlabel('Actual Irradiance (W/m²)')
    plt.ylabel('Predicted Irradiance (W/m²)')
    plt.tight_layout()
    
    # Save the overall scatter plot
    overall_scatter_filename = 'actual_vs_predicted_overall.png'
    overall_scatter_path = os.path.join(run_folder, overall_scatter_filename)
    plt.savefig(overall_scatter_path)
    plt.close()
    print(f"Saved overall scatter plot: {overall_scatter_path}")

In [44]:
evaluate_model(model, test_data, sequence_length=60)

Created run-specific directory at 'evaluation_plots/run_20241119_220812'.
Created 65 sequences for evaluation.
Evaluation split: 65 samples.
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Generated predictions for the test set.
5-Minute Ahead Prediction - RMSE: 165.72, MAE: 162.16, R²: -46.01
Saved plot: evaluation_plots/run_20241119_220812/5_min_ahead_prediction.png
Saved scatter plot: evaluation_plots/run_20241119_220812/actual_vs_predicted_5_min_ahead.png
15-Minute Ahead Prediction - RMSE: 129.65, MAE: 123.52, R²: -13.16
Saved plot: evaluation_plots/run_20241119_220812/15_min_ahead_prediction.png
Saved scatter plot: evaluation_plots/run_20241119_220812/actual_vs_predicted_15_min_ahead.png
30-Minute Ahead Prediction - RMSE: 127.72, MAE: 118.12, R²: -6.35
Saved plot: evaluation_plots/run_20241119_220812/30_min_ahead_prediction.png
Saved scatter plot: evaluation_plots/run_20241119_220812/actual_vs_predicted_30_min_ahead.png
60-Minute Ahead Prediction - RMSE: 57.88, MAE: 47.50, R²: -0.08
Saved p